In [36]:
from Crypto.Cipher import Blowfish, PKCS1_OAEP, AES
from Crypto.PublicKey import RSA
from Crypto.Util.Padding import pad, unpad
from binascii import hexlify, unhexlify
import hashlib
import json
import string
import random
from stegano import lsb
from datetime import datetime
from Crypto.Random import get_random_bytes
from Crypto.Cipher import PKCS1_OAEP, AES
from Crypto.PublicKey import RSA
import os

In [37]:
def key_generator(size, case="default", punctuations="required"):
    if case == "default" and punctuations == "required":
        return ''.join(random.choices(string.ascii_uppercase + string.ascii_lowercase + string.digits + string.punctuation, k=size))
    elif case == "upper-case-only" and punctuations == "required":
        return ''.join(random.choices(string.ascii_uppercase + string.digits + string.punctuation, k=size))
    elif case == "lower-case-only" and punctuations == "required":
        return ''.join(random.choices(string.ascii_lowercase + string.digits + string.punctuation, k=size))
    elif case == "default" and punctuations == "none":
        return ''.join(random.choices(string.ascii_uppercase + string.digits + string.ascii_lowercase, k=size))
    elif case == "lower-case-only" and punctuations == "none":
        return ''.join(random.choices(string.ascii_lowercase + string.digits, k=size))
    elif case == "upper-case-only" and punctuations == "none":
        return ''.join(random.choices(string.ascii_uppercase + string.digits, k=size))

In [38]:
def auto_test(filename):
    filename_parts = filename.split('.')
    if len(filename_parts) != 2:
        print(f"Invalid filename format: {filename}")
        return

    file_name = filename_parts[0]
    file_ext = filename_parts[1]

    base_directory = r'C:\Users\HP\Desktop\Isogeny'

    # Create the logs directory if it doesn't exist
    logs_directory = os.path.join(base_directory, 'logs')
    if not os.path.exists(logs_directory):
        os.makedirs(logs_directory)
    
    # Specify the complete path for the log file
    log_file_path = os.path.join(logs_directory, 'encryption-log.txt')
    


    with open(f'{base_directory}/test_files/{file_name}.{file_ext}', 'rb') as file:
        plaintext = file.read()

    log_plaintext_length = len(hexlify(plaintext))

    password = 'hyenc2021'

    log_password_length = len(password)

    log_start_time = datetime.now()

    hash = hashlib.sha1()
    hash.update(password.encode())
    password_encryption_cipher = AES.new(hash.hexdigest()[:16].encode(), AES.MODE_CBC, iv='16bitAESInitVect'.encode())

    keys_iv = {}

    blowfish_key = key_generator(size=16).encode()
    blowfish_cipher = Blowfish.new(blowfish_key, Blowfish.MODE_CBC)

    blowfish_ciphertext = blowfish_cipher.encrypt(pad(plaintext, Blowfish.block_size))

    keys_iv['blowfish_iv'] = hexlify(blowfish_cipher.iv).decode()
    keys_iv['blowfish_key'] = hexlify(blowfish_key).decode()

    rsa_key = RSA.generate(2048)
    rsa_private_key = rsa_key
    rsa_public_key = rsa_key.publickey()

    cipher_rsa = PKCS1_OAEP.new(rsa_public_key)
    rsa_plaintext = blowfish_ciphertext

    rsa_ciphertext = bytearray()
    for i in range(0, len(rsa_plaintext), 190):
        rsa_ciphertext.extend(cipher_rsa.encrypt(rsa_plaintext[i:i + 190]))

    keys_iv['rsa_n'] = rsa_private_key.n
    keys_iv['rsa_e'] = rsa_private_key.e
    keys_iv['rsa_d'] = rsa_private_key.d

    aes_key = key_generator(size=16).encode()
    aes_cipher = AES.new(aes_key, AES.MODE_CBC)
    aes_plaintext = rsa_ciphertext

    aes_ciphertext = aes_cipher.encrypt(pad(aes_plaintext, AES.block_size))

    ciphertext = aes_ciphertext

    with open(f'{base_directory}/test_files/{file_name}_hyenc.encrypted', 'wb') as file:
        file.write(ciphertext)

    log_ciphertext_length = len(ciphertext)

    keys_iv['aes_iv'] = hexlify(aes_cipher.iv).decode()
    keys_iv['aes_key'] = hexlify(aes_key).decode()

    encrypted_keys_and_iv = hexlify(password_encryption_cipher.encrypt(pad(json.dumps(keys_iv).encode(), AES.block_size)))

    lsb_stegano_image = lsb.hide(f"{base_directory}/test_files/cover_image.png", encrypted_keys_and_iv.decode())
    lsb_stegano_image.save(f"{base_directory}/test_files/stego_image.png")

    log_end_time = datetime.now()

    log_duration = str(log_end_time - log_start_time)

    with open(f'{base_directory}/logs/encryption-log.txt', 'a+') as log_file:
        log_file.write("\n| " + str(log_plaintext_length)
                       + "          | " + str(log_ciphertext_length)
                       + "          | " + str(log_password_length)
                       + "         | " + log_start_time.strftime("%H:%M:%S")
                       + "   | " + log_end_time.strftime("%H:%M:%S")
                       + "  | " + str(log_duration)
                       + " |"
                       )

    print('File Encryption Complete!')


In [39]:
def decryption(filename):
    # Split the filename into name and extension
    filename_parts = filename.split('.')
    if len(filename_parts) != 2:
        print(f"Invalid filename format: {filename}")
        return

    file_name = filename_parts[0]
    file_ext = filename_parts[1]

    base_directory = r'C:\Users\HP\Desktop\Isogeny'

    # Password for Keys
    password = 'hyenc2021'

    with open(f'{base_directory}/test_files/{file_name}_hyenc.encrypted', 'rb') as file:
        ciphertext = file.read()

    log_password_length = len(password)
    log_ciphertext_length = len(ciphertext)

    log_start_time = datetime.now()

    unhide_encrypted_keys_and_iv = lsb.reveal(f"{base_directory}/test_files/stego_image.png").encode()

    hash = hashlib.sha1()
    hash.update(password.encode())
    password_decryption_cipher = AES.new(hash.hexdigest()[:16].encode(), AES.MODE_CBC, iv='16bitAESInitVect'.encode())

    decrypted_keys_iv = json.loads(unpad(password_decryption_cipher.decrypt(unhexlify(unhide_encrypted_keys_and_iv)), AES.block_size))

    aes_cipher = AES.new(unhexlify(decrypted_keys_iv['aes_key']), AES.MODE_CBC, iv=unhexlify(decrypted_keys_iv['aes_iv']))
    aes_plaintext = aes_cipher.decrypt(ciphertext)

    aes_plaintext = unpad(aes_plaintext, AES.block_size)

    rsa_private_key = RSA.construct((int(decrypted_keys_iv['rsa_n']), int(decrypted_keys_iv['rsa_e']), int(decrypted_keys_iv['rsa_d'])))
    cipher_rsa = PKCS1_OAEP.new(rsa_private_key)
    rsa_plaintext = bytearray()
    for i in range(0, len(aes_plaintext), 256):
        rsa_plaintext.extend(cipher_rsa.decrypt(aes_plaintext[i:i + 256]))

    blowfish_cipher = Blowfish.new(unhexlify(decrypted_keys_iv['blowfish_key']), Blowfish.MODE_CBC, iv=unhexlify(decrypted_keys_iv['blowfish_iv']))
    blowfish_plaintext = blowfish_cipher.decrypt(rsa_plaintext)

    blowfish_plaintext = unpad(blowfish_plaintext, Blowfish.block_size)

    with open(f'{base_directory}/test_files/{file_name}_decrypted.{file_ext}', 'wb') as file:
        file.write(blowfish_plaintext)

    log_end_time = datetime.now()
    log_duration = str(log_end_time - log_start_time)

    with open(f'{base_directory}/logs/decryption-log.txt', 'a+') as log_file:
        log_file.write("\n| " + str(log_ciphertext_length)
                        + "          | " + str(log_password_length)
                        + "         | " + log_start_time.strftime("%H:%M:%S")
                        + "   | " + log_end_time.strftime("%H:%M:%S")
                        + "  | " + str(log_duration)
                        + " |"
                        )

    print('File Decryption Complete!')

In [40]:


if __name__ == "__main__":
    filename_to_decrypt = "sample.txt"  # Replace with the actual filename
    auto_test(filename_to_decrypt)
    decryption(filename_to_decrypt)


File Encryption Complete!
File Decryption Complete!
